# Conversational AI (25/26) Assignment 4: Prompting
This notebook uses Qwen to generate a dialogue utterance by providing various instructions. 
The dataset used in the assignment is the one from [DSTC11's task 5](https://github.com/lkra/dstc11-track5/blob/main/data), same as assignments 1-3.


## Assignment
In this assignment, we will load a large language model (Qwen) and ask it to generate a dialogue response using different instructions and examples. 
The notebook compares three different prompting techniques:
* Zero-shot prompting
* Few-shot prompting
* Chain-of-Thought prompting

For each of these techniques, we explore how to interact with the model, design prompts, and interpret the response.

**NOTE:** In addition to the most common approaches, there are other, less widely adopted prompting techniques, such as *CARP* (Chain-of-Thought with Retrieval Augmented Prompting) or *LPP* (Least-to-Most Prompting). It is important to remember that prompting is a flexible strategy that can be tailored to the specific needs of classification tasks.

This notebook covers the following aspects:

1. Implementation of three popular prompting methods (from lecture 4)
2. Evaluation with multiple metrics (from assignment 3)
3. Analyzing the model behavior across configurations

**Assignment steps:**

First, review this notebook to ensure you understand its purpose.

Throughout the notebook, you will find questions you need to answer to complete the assignment. These are both coding questions and questions that evaluate your understanding of the data, the process, and the model behavior. These questions are indicated as <span style="background:blue">__Q#__</span>. This notebook contains 9 such questions and tasks:

1. Complete the prompting function and run zero- and few-shot prompting with Qwen2.5-3B and Qwen3-1.7B. **(Q1, Q2)**
2. Compare zero-and few-shot performance of Qwen2.5-3B and Qwen3-1.7B. **(Q3)**
3. Complete the chain-of-thought code. **(Q4)**
4. Answer the questions about the comparison of the methods. **(Q5 - Q9)**


**Submission:**
Please submit your code (as a Kaggle notebook) on Canvas by **24th November 23:59**.

**Grading:** The assignment is graded with a pass/fail grade.

## Reminders about Kaggle notebooks
1. Pay attention to usage statistics, especially memory, CPU, and GPU
2. Pay attention to the quota of GPU (measured in hours)
3. "Turn OFF" the internet after each session. Turn on the internet when starting a session.
4. "Turn off" the accelerator after each use. Turn the accelerator on when starting a session.


## Ready?
Let's get started! There are four preparations we will start with:
1. installing packages
2. importing relevant packages
3. setting up the directory with the task data
4. defining auxiliary LLM functions

## Installation

The Kaggle notebooks use a Python 3 environment, and they are already "pre-loaded" with various analytic Python packages, like Json, Pandas, and Numpy. If you are curious, you can see the package definition in [this repository](https://github.com/kaggle/docker-python).

We will install a few specialized packages for evaluation and for working with language models.

Let's just double check that Python is set up and we are using a relatively new version (like 3.10):

In [1]:
!python --version

Python 3.14.0


In [2]:
# To use BLEU and ROUGE
!pip install sacrebleu
!pip install rouge

# To use BERTSCORE
!pip install evaluate
!pip install bert_score

# To use ْQwen
!pip install transformers

# To calculate cosine similarities between sentences
!pip install sentence-transformers

## Imports

The following code loads several standard packages, packages for evaluation, and packages for working with LLMs.

In [3]:
import numpy as np 
import json
import os
import shutil
import subprocess

from sacrebleu.metrics import BLEU
from rouge import Rouge

from evaluate import load

from sentence_transformers import CrossEncoder

from transformers import AutoModelForCausalLM, AutoTokenizer

bleu_scorer = BLEU()
rouge_scorer = Rouge()
bertscore = load("bertscore")


/home/song0409/anaconda3/envs/env_cai/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Clone the data/task repository

In [4]:
#Localisation of working directory

work_dir ="/home/song0409/Desktop/CAI"

In [5]:
def setup_repo(repo_url: str, repo_name: str, work_dir: str = "/kaggle/working"):
    os.chdir(work_dir)
    
    # Remove repo if it exists
    if os.path.exists(os.path.join(work_dir, repo_name)):
        shutil.rmtree(os.path.join(work_dir, repo_name))
    
    # Clone repo
    subprocess.run(["git", "clone", repo_url], check=True)
    
    # Move into repo/data
    os.chdir(os.path.join(repo_name, "data"))


setup_repo("https://github.com/lkra/dstc11-track5.git", "dstc11-track5", work_dir)

Cloning into 'dstc11-track5'...


In [6]:
## List all files in the current directory iteratively:
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./output_schema.json
./README.md
./knowledge_aug_domain_reviews.json
./knowledge_aug_reviews.json
./knowledge.json
./val/labels.json
./val/logs.json
./test/labels.json
./test/logs.json
./train/logs_bkp.json
./train/labels.json
./train/logs.json
./train/bkp/labels.json
./train/bkp/logs.json


There are two files that are vital to this notebook. `Logs` contains the conversation logs, whereas `labels` are the ground-truth responses by humans. Feel free to inspect the contents of these files at this point.

In [7]:
with open('train/logs.json', 'r') as f:
    contexts=json.load(f)

with open('train/labels.json', 'r') as f:
    labels=json.load(f)

## Helper functions

In this section, we define several utility functions that will be used later in the assignment. You may need to get back to this section for some of the assignment steps.

### Evaluation functions

We will start by defining wrapper functions for our evaluation metrics. ROUGE, BLEU, and Bertscore are implemented fully in this notebook already. We suggest you to spend a few minutes to call these functions with some sample inputs to understand how they work.

In [8]:
def compute_rouge(hypothesis, reference):
    """
    Returns the prediction and the reference for a single dialogue, returns their ROUGE score.
    """
    score = rouge_scorer.get_scores(
          hyps=hypothesis,
          refs=reference,
      )
    return score[0]["rouge-l"]["f"]

def compute_bleu(hypothesis, reference):
    """
    Returns the prediction and the reference for a single dialogue, returns their BLEU score.
    """
    score = bleu_scorer.sentence_score(
        hypothesis=hypothesis,
        references=[reference],
      )
    return score.score/100 # sacreBLEU gives the score in percent

def compute_bertscore(predictions, references):
    """
    Receives two lists of strings with equal length. Returns their pairwise Bertscores (precision, recall, F1 score).
    """
    results = bertscore.compute(predictions=predictions, references=references, lang="en")
    return results

### LLM functions


 * **get_qwen3_model** and **get_qwen2_model:** Load the Qwen models and assign them to global variables.
 * **query_qwen:** Get the output of the models for a given prompt.


In [9]:
_qwen3_model = None
_qwen3_tokenizer = None

_qwen2_model = None
_qwen2_tokenizer = None


def get_qwen3_model():
    global _qwen3_model, _qwen3_tokenizer
    if _qwen3_model is None or _qwen3_tokenizer is None:
        model_name = "Qwen/Qwen3-1.7B"
        _qwen3_tokenizer = AutoTokenizer.from_pretrained(model_name)
        _qwen3_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"
        )
    return _qwen3_model, _qwen3_tokenizer


def get_qwen2_model():
    global _qwen2_model, _qwen2_tokenizer
    if _qwen2_model is None or _qwen2_tokenizer is None:
        model_name = "Qwen/Qwen2.5-3B-Instruct"
        _qwen2_tokenizer = AutoTokenizer.from_pretrained(model_name)
        _qwen2_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"
        )
    return _qwen2_model, _qwen2_tokenizer

For Qwen3-1.7B, we set the *enable_thinking* parameter to *false* because enabling it makes the model take much longer to produce an answer, and we don’t need that level of detailed reasoning for this task

In [ ]:
def query_qwen(messages, model_name):
    
    if model_name == "Qwen3-1.7B":
        model, tokenizer = get_qwen3_model()

        text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
        )
        
    elif model_name == "Qwen2.5-3B":
        model, tokenizer = get_qwen2_model()

        text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
        )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2000
    )

    output_ids = generated_ids[0][model_inputs.input_ids.shape[1]:]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return output



Next, we define functions to prepare the examples.

 * **get_examples:** retrieves a subset of context–label pairs with valid targets from the dataset.

 * **prepare_examples:** formats the retrieved pairs into structured textual demonstrations suitable for few-shot prompting.


In [11]:
def prepare_examples(chat, gt_label):
    """
    Transforms a chat into a demonstration example for an LLM. 
    Returns a demonstration sample for a single prior chat.
    """
    demo=""
    for ut in chat:
        role = "user" if ut['speaker']=="U" else "system"
        demo+= f'{role}: {ut["text"]}\n'
    demo+=f'Response: {gt_label}\n'
    return demo


def get_examples(contexts, labels, n_examples):
    """
    Get `n_examples` chats with labels, ignoring those where we have no response. 
    To make sure that we don't pollute the evaluation, 
    we obtain examples from the end of the dataset. 
    Returns a list of example chats and a list of example labels.
    """
    example_chats=[]
    example_labels=[]

    for c, l in zip(reversed(contexts), reversed(labels)):
        if l['target']:
            example_chats.append(c)
            example_labels.append(l)
            if len(example_chats)==n_examples:
                break
    return example_chats, example_labels
    


<span style="background:blue">__Q1:__ Complete the `prepare_messages` function, which is necessary for the prompting to work.</span>

**prepare_messages:** this function assembles the complete prompt for the model, combining the initial instruction, training examples, and the history of the current conversation. You need to define the initial instruction and training examples, and ensure that each turn in the conversation history is properly mapped to a message with the correct role (user or assistant) and its corresponding content.

In [12]:
def prepare_messages(history, example_chats=[], example_labels=[]):
    """
    Prepare the messages that the model will get as input. 
    Considers the primary model instruction, demonstrations for few-shot settings, 
    and the history of the current chat. Returns the list of messages.
    """
    
    #TODO: write a suitable instruction for the model.
    instruction=f'Your role is a chatbot for a hotel booking website. You are highly capable of recognizing the user"s requirement, constraints and conditions of booking of the hotel.'

    if len(example_chats): # for few-shot only
        #TODO: Complete this function for few-shot
        instruction+=f'Pay attention to the way how the system(speaker:"S") answers to the user and keep in mine that the goal is to make a booking for the user.'
        example_counter = 1
        for chat, label in zip(example_chats,example_labels):
            formatted_example=prepare_examples(chat,label)
            instruction+=f'Example {example_counter}:\n{formatted_example}\n'
            example_counter += 1
    
    # Construct dialogue history
    messages=[]
    messages.append({"role": "system", "content": instruction})
    for ut in history:
        role = "user" if ut['speaker']=="U" else "system"
        messages.append({"role": role, "content": ut['text']})

    return messages

## Zero- and few-shot model run and evaluation

<span style="background:blue">__Q2:__ Use the helper functions to run Qwen2.5-3B and Qwen3-1.7B  in this block. Make sure your code shows both their zero- and few-shot evaluations (adapt the code and add functions where necessary).</span>

* Construct the prompt (including demonstrations when applicable).

* Execute each model on `n_dialogues` = 100.

* Evaluate the model outputs and report metrics separately for each model and regime (zero-shot vs. few-shot).

* **evaluate_answers:** this function evaluates how similar a model’s output (model_response) is to a ground truth answer (gt_response) using the metrics ROUGE and BLEU.

In [13]:
def evaluate_answers(model_response, gt_response):
    rouge=compute_rouge(model_response, gt_response)
    bleu=compute_bleu(model_response, gt_response)

    print("MODEL:", model_response)
    print("GROUND TRUTH:", gt_response)
    print("ROUGE:", rouge, "BLEU:", bleu)

    print()

    return rouge, bleu
    

* **final_evaluation:** performs an evaluation of the model’s performance, reporting the average BLEU, ROUGE, and BERTScore.

In [14]:
def final_evaluation(all_bleus, all_rouges, all_predictions, all_references):
    print("Mean BLEU", np.mean(all_bleus))
    print("Mean ROUGE", np.mean(all_rouges))
    print("BERTscore", np.mean(compute_bertscore(all_predictions, all_references)['f1']))

* **run_pipeline:** this function iterates over the dialogs until reaching *n_dialogues*. It builds the prompt (zero-shot or few-shot) and generates the model's response (*query_qwen*). Then, it evaluates that response with ROUGE and BLEU metrics. Finally, it saves predictions, references, and metrics in lists and prints the progress and results of each dialog.

In [15]:
def run_pipeline(n_dialogues, n_examples, model_name):
    if n_examples>0:
        example_chats, example_labels=get_examples(contexts, labels, n_examples)
        print("Example chat", example_chats)
    
    all_bleus=[]
    all_rouges=[]
    all_predictions=[]
    all_references=[]
    
    counter_dialogues = 0
    idx_dialogue = 0

    while counter_dialogues < n_dialogues:
        ## Get a valid dialogue
        chat = contexts[idx_dialogue]
        if 'response' not in labels[idx_dialogue]:
            idx_dialogue += 1
            continue
        gt_response = labels[idx_dialogue]['response']
        counter_dialogues += 1
        idx_dialogue += 1
        
        print("Processing dialogue", counter_dialogues)
    
        ## prepare messages
        messages = prepare_messages(chat, example_chats, example_labels) if n_examples > 0 else prepare_messages(chat)
        
        ## get the model's output
        model_response = query_qwen(messages, model_name)
            
        ## Evaluation
        rouge, bleu = evaluate_answers(model_response, gt_response)
    
        ## save the results
        all_predictions.append(model_response)
        all_references.append(gt_response)
        all_bleus.append(bleu)
        all_rouges.append(rouge)

    final_evaluation(all_bleus, all_rouges, all_predictions, all_references)
    

Let's print all our metrics for this configuration:

In [16]:
n_dialogues = 100
n_examples = 0 # 0 for zero-shot; >0 for few-shot
model_name = "Qwen3-1.7B" ## "Qwen2.5-3B"
# model_name = "Qwen2.5-3B"

run_pipeline(n_dialogues, n_examples, model_name)

Processing dialogue 1


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.60it/s]
It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: Yes, the Bridge Guesthouse is known for its comfortable beds. They offer a cozy and welcoming environment, and the beds are designed to provide a restful night's sleep. If you have any specific preferences or need further assistance with your reservation, feel free to let me know!
GROUND TRUTH: The Bridge Guest House is known for having pretty uncomfortable beds according to most guests. Only one guest found it to be comfortable.
ROUGE: 0.2187499954882813 BLEU: 0.03453783827607845

Processing dialogue 2


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: Yes, the Curry Garden offers an outdoor dining area with a beautiful garden setting. They also have a private terrace for a more secluded experience. Would you like me to check availability for you?
GROUND TRUTH: The Curry Garden has a patio outside that guests say they enjoy. Do you want to make a reservation now?
ROUGE: 0.1999999952880001 BLEU: 0.027410126102418365

Processing dialogue 3


...

MODEL: Yes, the Acorn Guest House is located in the heart of Cambridge, near the city center, and offers a convenient and comfortable stay. It is well positioned for exploring the city's attractions, including museums, historic sites, and cafes. The location is ideal for both leisure and business travelers. Would you like to proceed with booking this place?
GROUND TRUTH: While the location of the Acorn Guest House can be considered safe and comfortable, it's far from the center of the city, which leaves guests with mixed opinions.
ROUGE: 0.22222221777777784 BLEU: 0.05513586763135976

Processing dialogue 100


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: I'd be happy to help! Prezzo is known for its excellent customer service, friendly staff, and great atmosphere. They're a popular choice for moderately priced Italian food on the west side of town. Would you like more details about their menu, hours, or any other information?
GROUND TRUTH: I found two people that say the Prezzo has good service, and one person that said the service was just okay.
ROUGE: 0.12499999582519546 BLEU: 0.02108443790153119

Mean BLEU 0.046466411245933356
Mean ROUGE 0.2075062569381318


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTscore 0.8713123708963394


## zero-shot
    Mean BLEU 0.042212767287859145
    Mean ROUGE 0.20143690509066758
    BERTscore 0.870943506360054

## 1-shot
    Mean BLEU 0.021758916843830415
    Mean ROUGE 0.14957201795045896
    BERTscore 0.8131398820877075


## 3-shot 
    Mean BLEU 0.012506892229175188
    Mean ROUGE 0.13231982105963921
    BERTscore 0.7964058488607406

## 5-shot
    Mean BLEU 0.014724266495214893
    Mean ROUGE 0.13881729033579884
    BERTscore 0.8017561084032059



<span style="background:blue">__Q3:__ List the results you obtain for both models for zero-shot prompting on the automated metrics. Then, briefly reflect: how high are the scores, which model does better, and why may that be?</span>

```
The zero shot model with an instruction "Your role is a chatbot for a hotel booking website. You are highly capable of recognizing the user"s requirement, constraints and conditions of booking of the hotel." outperforms the few-show models by average 2 times in Mean BLUE score, 1.5 times in Mean ROUGE score and 1.1 times in BERTscore. However, it is counterintuitive result as we assume that the few-shot approach would improve the quality of response by providing clear examples to the model. After researches on the conversation examples regarding this unexpected result all the information provided by system was overly aligned to the examples that provided, which leads the model to adapt the perspectives and structure of the examples even if what the user want to be informed was appart from the corresponding approach. Therefore, the other many cases where the user requirments are different and versatile, the model tends to miss the words that match the semantics as well as n-grams on GROUND TRUTH. 
```

## Chain-of-Thought
<span style="background:blue">__Q4:__ Incorporate chain-of-thought prompting in the instruction field. Use the provided instruction text below. Adapt the response parsing code to evaluate only the generation within `<output>` tags.</span>

What you need to do:

- Create a new function, similar to prepare_messages, that uses the new prompt.
- Create another function, similar to run_pipeline, that calls this new prepare_messages function.
- Run the pipeline with both models in a zero-shot setting.
- Extract the output from the <output> tag.
- Check whether the models follow the requested format and examine how their outputs differ.
- Get new results

In [33]:
# from https://ollama.com/zcohn/llama3.1-uncensored-cot/blobs/54b94c1841a9
chain_of_thought_instruction="""
1. Begin with a <thinking> section. 
2. Inside the thinking section: 
    a. Briefly analyze the question and outline your approach. 
    b. Present a clear plan of steps to solve the problem. 
    c. Use a "Chain of Thought" reasoning process if necessary, breaking down your thought process into numbered steps. 
3. Include a <reflection> section for each idea where you: 
    a. Review your reasoning. 
    b. Check for potential errors or oversights. 
    c. Confirm or adjust your conclusion if necessary. 
4. Be sure to close all reflection sections. 
5. Close the thinking section with </thinking>. 
6. Provide your final answer in an <output></output> section.

Remember:
Always use these tags in your responses. 
Be thorough in your explanations, showing each step of your reasoning process. 
Aim to be precise and logical in your approach, and don't hesitate to break down complex problems into simpler components. 
Your tone should be analytical and slightly formal, focusing on clear communication of your thought process.

Remember: 
<thinking> and <reflection> and <output> MUST be tags and must be closed at their conclusion. 
Make sure all <tags> are on separate lines with no other text. Do not include other text on a line containing a tag.
You MUST provide your final answer inside the <output></output> tags. 

Provide your output in the following format:
<thinking>
[your initial thinking goes here]
</thinking>

<reflection>
[Review your previous reasoning, either confirm or adjust]
</reflection>

<output>
[Provide your final answer here.]
</output>

As shown in the example above, all three tags must be there, and inside them, you must fill in the requested information.
DO not provide any information outside the tags. All the information must go in the suitable tag.

"""

# Your solution here.

import re

def new_prepare_messages(history):
    """
    New prepare_messages function with a new instruction.
    """
    
    instruction=chain_of_thought_instruction
    
    # Construct dialogue history
    messages=[]
    messages.append({"role": "system", "content": instruction})
    for ut in history:
        role = "user" if ut['speaker']=="U" else "system"
        messages.append({"role": role, "content": ut['text']})

    return messages

def response_adapter(model_response):
    # This pass the model response to extract the only response in <output>

    pattern = r"<output>(.*?)</output>"
    output = "".join(re.findall(pattern,model_response,re.DOTALL))


    return output


def new_run_pipeline(n_dialogues, model_name):
    
    all_bleus=[]
    all_rouges=[]
    all_predictions=[]
    all_references=[]
    
    counter_dialogues = 0
    idx_dialogue = 0

    while counter_dialogues < n_dialogues:
        ## Get a valid dialogue
        chat = contexts[idx_dialogue]
        if 'response' not in labels[idx_dialogue]:
            idx_dialogue += 1
            continue
        gt_response = labels[idx_dialogue]['response']
        counter_dialogues += 1
        idx_dialogue += 1
        
        print("Processing dialogue", counter_dialogues)
    
        ## prepare messages
        messages = new_prepare_messages(chat)
        
        ## get the model's output
        model_response = query_qwen(messages, model_name)
        
        ## extract the only output
        model_output = response_adapter(model_response)

        if model_output == "":
            print("No response!")
            continue

        ## Evaluation
        rouge, bleu = evaluate_answers(model_output, gt_response)
        # rouge, bleu = evaluate_answers(model_response, gt_response)
    
        ## save the results
        all_predictions.append(model_output)
        all_references.append(gt_response)
        all_bleus.append(bleu)
        all_rouges.append(rouge)

    final_evaluation(all_bleus, all_rouges, all_predictions, all_references)


In [19]:
new_run_pipeline(n_dialogues, model_name)

Processing dialogue 1


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
The Bridge Guesthouse is located in the south and offers free parking and free internet. While the beds are likely comfortable, there is no specific mention of bed comfort in the provided data. I recommend contacting the guesthouse directly for more details about the beds and any additional features that might help with your back issues.

GROUND TRUTH: The Bridge Guest House is known for having pretty uncomfortable beds according to most guests. Only one guest found it to be comfortable.
ROUGE: 0.14492753188825888 BLEU: 0.01709714016749041

Processing dialogue 2


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
The Curry Garden is an expensive Indian restaurant in the city that offers outdoor dining. However, I don't have specific information confirming if it has an outdoor dining area. Would you like me to proceed with the reservation, or would you prefer a different restaurant? 

GROUND TRUTH: The Curry Garden has a patio outside that guests say they enjoy. Do you want to make a reservation now?
ROUGE: 0.2711864363114048 BLEU: 0.03755967938178985

Processing dialogue 3


...

It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
The hotel "Finches" could not be found in the system. However, the "Cambridge Belfry" is a 4-star hotel in the same area and price range. Would you like me to check if the staff there are nice and welcoming? If so, I can provide more details about the staff and proceed with the booking.

GROUND TRUTH: Some of the guests of The Cambridge Belfry said that the staff was unfriendly and rude while others say that the staff was polite and kind.
ROUGE: 0.12499999582519546 BLEU: 0.022098435817064804

Processing dialogue 9


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
The Lovell Lodge is known for its friendly staff and is a great choice for a moderate-priced stay with free WiFi and free parking. If you're looking for a comfortable and welcoming environment, the Lovell Lodge is a solid option. For more details about the staff, you can contact the hotel directly or check online reviews. I recommend considering this option based on your preferences.

GROUND TRUTH: Lovell Lodge has friendly staff that is kind and respectful. Do you want to book a reservation there?
ROUGE: 0.23188405411468183 BLEU: 0.017624509187982147

Processing dialogue 10


KeyboardInterrupt: 

## Questions

**Pick the first 100 samples for automated metric analysis and use them to answer the questions below. When asked for manual evaluation, do this on 5 dialogues.**

Please answer each question with a single paragraph, consisting of 2-5 sentences. Include examples to support your answers. Remember to give examples and include results when describing your findings!



<span style="background:blue">__Q5:__ Test Qwen3-1.7B and Qwen2.5-3B in few-shot setting with 1, 3, and 5 examples. What do you observe in terms of the evaluation metrics BLEU, ROUGE, [Bertscore](https://huggingface.co/spaces/evaluate-metric/bertscore)? For the remaining questions and experiments, proceed with the language model (Qwen3-1.7B or Qwen2.5-3B) that yielded better scores on this evaluation.</span>

```
1. Qwen3-1.7B

    zero-shot
        Mean BLEU 0.042212767287859145
        Mean ROUGE 0.20143690509066758
        BERTscore 0.870943506360054

    1-shot
        Mean BLEU 0.021758916843830415
        Mean ROUGE 0.14957201795045896
        BERTscore 0.8131398820877075


    3-shot 
        Mean BLEU 0.012506892229175188
        Mean ROUGE 0.13231982105963921
        BERTscore 0.7964058488607406

    5-shout
        Mean BLEU 0.014724266495214893
        Mean ROUGE 0.13881729033579884
        BERTscore 0.8017561084032059

2. Qwen2.5-3B

    zero-shot
        Mean BLEU 0.03552609006705259
        Mean ROUGE 0.1820990779324724
        BERTscore 0.8671524453163147

    1-shot
        Mean BLEU 0.03004565929506359
        Mean ROUGE 0.1698382685711549
        BERTscore 0.835916965007782

    3-shot
        Mean BLEU 0.021867288699055017
        Mean ROUGE 0.14546228435740732
        BERTscore 0.8168431138992309

    5-shot
        Mean BLEU 0.024566090782345364
        Mean ROUGE 0.15470095814685936
        BERTscore 0.8190595406293869


Considering the results of experiements, we observe that the Qwen3-1.7B outperforms the other in all different parameters.
```

<span style="background:blue">__Q6:__ Now define three quality criteria that you will apply to evaluate manually, by consulting papers by [Deriu et al.](https://link.springer.com/article/10.1007/s10462-020-09866-x) and by [Howcroft et al.](https://aclanthology.org/2020.inlg-1.23/). If you did this in assignment 3, you can just reuse the metrics here. Analyze the generation with 0, 1, and 5 demonstrations using these three manual metrics, for 5 dialogues. Which of the methods seems to be of the highest quality according to your manual assessment?</span>

```
# Your answer here
```

<span style="background:blue">__Q7:__  Now compare your manual evaluation from Q6 to the results you obtained with the automatic metrics. What do you observe?</span>


```
# Your answer here
```

<span style="background:blue">__Q8:__ Compare zero-shot chain-of-thought performance with that of standard zero-shot (using both the automatic metrics and manual judgment). What do you observe?</span>




## 1. zero shot standard

    Overall scores upon 100 dialogues
    Mean BLEU 0.046466411245933356
    Mean ROUGE 0.2075062569381318
    BERTscore 0.8713123708963394


## 2. zero shot with CoT 

    Overall scores upon 100 dialogues
    Mean BLEU 0.03500307343127906
    Mean ROUGE 0.18746475794246212
    BERTscore 0.8193042474923674



## 3. Evaluation on 5 dialogue

### Processing dialogue 1

    GROUND TRUTH: The Bridge Guest House is known for having pretty uncomfortable beds according to most guests. Only one guest found it to be comfortable.

    STANDARD MODEL: Yes, the Bridge Guesthouse is known for its comfortable beds. They offer a cozy and welcoming environment, and the beds are designed to provide a restful night's sleep. If you have any specific preferences or need further assistance with your reservation, feel free to let me know!

    ROUGE: 0.2187499954882813 BLEU: 0.03453783827607845

    COT MODEL: Yes, the Bridge Guesthouse provides comfortable beds. The guesthouse is known for its supportive mattress and comfortable sleeping environment, which is ideal for individuals with back issues. If you'd like, I can assist with booking a reservation and confirming the details. Let me know if you need further assistance!   

    ROUGE: 0.21538461090650898  BLEU: 0.03178743908080704


        => The both models failed to generate a correct answer, even though the metrics show relative high scores due to high n-grams. 

----------------------------------------------------------------------------------------------------------------------------------------
### Processing dialogue 5

    GROUND TRUTH: Yes they do have an incredible view which is amazing .The location is also nice and they do have amazing décor.

    STANDARD MODEL: I'm sorry, but I don't have information about Kymmoy's menu or offerings. Could you please provide more details about what you're looking for? For example, are you looking for a specific type of Asian cuisine, a particular restaurant, or a specific price range? I'd be happy to help you find the best option for your business meeting.

    ROUGE: 0.031249996250000453 BLEU: 0.007328733472514606

    COT MODEL: 
    I don't have specific information about whether Kymmoy offers good views. You might want to check directly with the restaurant or look for reviews online to confirm the view quality. Would you like me to book a table for you anyway?

    ROUGE: 0.07692307266272212  BLEU: 0.01176877973200725


        => COT model provided a correct answer that match the GROUND TRUTH while standard model has not generated relevant information, which is also represented by the distinct scores on metrics.
----------------------------------------------------------------------------------------------------------------------------------------

### Processing dialogue 10

    GROUND TRUTH: I only have one opinion and according to this opinion, the restaurant is nice inside, and it has a nice view.


    STANDARD MODEL: I'm not sure about the interior decor of the Good Luck Chinese Food Takeaway. I recommend checking their website or contacting them directly to get the most accurate information. Would you like me to look up any other details about this restaurant?

    ROUGE: 0.14035087274853814 BLEU: 0.012724686314753923

    COT MODEL: 
    Good Luck Chinese Food Takeaway is known for its clean and modern interior decor, which is typical of many food takeaways. The design is usually simple, with bright colors and a focus on functionality, which can make the dining experience pleasant. Therefore, it is likely to have an attractive interior decor.

    ROUGE: 0.12307691893964512 BLEU: 0.01139829220677002

        => It seems that the COT MODEL is overly confident when it comes to the case where only few or only one review is given, while the STANDARD model express the uncertainty of the information provided, which is the case that standard model behave more conservative manner.

----------------------------------------------------------------------------------------------------------------------------------------

### Processing dialogue 15

    GROUND TRUTH: The beds at the kirkwood are medium firm, and everyone pretty much says they feel amazing. Would you like to make a reservation?


    STANDARD MODEL: Yes, the beds at Kirkwood House are very comfortable and relaxing. They are designed to provide a restful and pleasant experience, making it ideal for a peaceful stay. Would you like me to check availability for you?

    ROUGE: 0.32727272240661165 BLEU: 0.07888036262523125

    COT MODEL: 
    Kirkwood House is a 4-star guest house with parking. It is known for its comfortable accommodations, including relaxing beds. If you'd like, I can provide more details about the beds or help you book it. Please let me know your check-in and stay dates, and the number of people for a confirmation.
    
    ROUGE: 0.14285713844489809 BLEU: 0.01680969088963095

        => The response from COT model generally matches the GROUND TRUTH and met the user's needs. However, the model provided too many details that are not mentioned in GROUND TRUTH, which lowered the scores due to the lost relevance and coherence while generating words.(same reason applied to dialogue 20) The STANDARD MODEL provided a correct answer that fairly matches the user's needs and successfully leads the user to the next step.
----------------------------------------------------------------------------------------------------------------------------------------

### Processing dialogue 20

    GROUND TRUTH: Yes, they do seem to have relaxing beds at Allenbell. Past guests have described them as being extremely comfortable.


    STANDARD MODEL: I don't have information about the specific beds at the Allenbell hotel. However, I can suggest a few options that fit your criteria:
        1. **The Grand Hotel** – Offers 4-star rating and free parking. Known for its comfortable rooms and relaxing atmosphere.
        2. **The Oakwood Inn** – Another 4-star option with free parking. It's located in a peaceful area and offers a cozy, relaxing stay.
        3. **The Harborview Hotel** – Provides free parking and is rated 4 stars. It's in a convenient location near amenities and offers a comfortable stay.
        Would you like me to check availability or provide more details about any of these options?

    ROUGE 0.14432989388457865 BLEU: 0.007861768009358769

    COT MODEL: 
    The Allenbell hotel is a 4-star hotel with free parking. While I don't have real-time data on bed quality, 4-star hotels generally offer comfortable accommodations. If you have any other questions or need further assistance, feel free to ask!
    
    ROUGE: 0.11320754268422945 BLEU: 0.012790871709798767

        => unlike the STANDARD MODEL, the COT MODEL provided a correct, tailered response to the user, including more details and the call for an action as well. However, the metric ROUGE of the COT MODEle was estimated lower than that of the STADARD MODEl due to the nature of evaluation based on n-grams.

----------------------------------------------------------------------------------------------------------------------------------------





<span style="background:blue">__Q9:__ Repeat the zero-shot experiments using two different temperature parameters: 0 and 1.
For each temperature, run the experiment on 5 dialogues, and repeat each experiment 3 times. Print both the generated outputs and the automatic evaluation metrics. </span>

Afterward, analyze the results:

- What do you observe for each temperature across different runs?
- Do you see consistency in the model’s answers or not?
- How do the metrics behave across runs?

In [31]:

def query_qwen_with_temperature(messages, model_name, set_temperature):
    
    if model_name == "Qwen3-1.7B":
        model, tokenizer = get_qwen3_model()

        text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
        )
        
    elif model_name == "Qwen2.5-3B":
        model, tokenizer = get_qwen2_model()

        text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
        )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2000,
        temperature = set_temperature
    )

    output_ids = generated_ids[0][model_inputs.input_ids.shape[1]:]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return output

def run_pipeline_with_temperature(n_dialogues, model_name,temperature):

    all_bleus=[]
    all_rouges=[]
    all_predictions=[]
    all_references=[]
    
    counter_dialogues = 0
    idx_dialogue = 0

    while counter_dialogues < n_dialogues:
        ## Get a valid dialogue
        chat = contexts[idx_dialogue]
        if 'response' not in labels[idx_dialogue]:
            idx_dialogue += 1
            continue
        gt_response = labels[idx_dialogue]['response']
        counter_dialogues += 1
        idx_dialogue += 1
        
        print("Processing dialogue", counter_dialogues)
    
        ## prepare messages
        messages = new_prepare_messages(chat)
        
        ## get the model's output
        model_response = query_qwen_with_temperature(messages, model_name, temperature)

        ## extract the only output
        model_output = response_adapter(model_response)

        if model_output == "":
            print("No response!")
            continue
            
        ## Evaluation
        rouge, bleu = evaluate_answers(model_output, gt_response)
    
        ## save the results
        all_predictions.append(model_output)
        all_references.append(gt_response)
        all_bleus.append(bleu)
        all_rouges.append(rouge)

    final_evaluation(all_bleus, all_rouges, all_predictions, all_references)
    


In [43]:
n_dialogues = 5
model_name = "Qwen3-1.7B"
Mean_BLUE = []
Mean_ROGUE = []
Mean_BERT= []

for i in range(2):
    print(f"Experiment with temperature {i}: \n")
    temperature = i +0.01 ## not possible to set 0
    print(f"\n"*5)

    for j in range(3):
        print(f"    Experiment {j+1} with temperature {i}: \n")
        run_pipeline_with_temperature(n_dialogues,model_name,temperature)
        print("--------------------------------------------------------------------------------------------------------------------------------------------\n")

Experiment with temperature 0: 







    Experiment 1 with temperature 0: 

Processing dialogue 1


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
The Bridge Guesthouse provides comfortable beds and is suitable for someone with back issues. I recommend booking a reservation to ensure a comfortable stay. Would you like me to book a reservation for you?

GROUND TRUTH: The Bridge Guest House is known for having pretty uncomfortable beds according to most guests. Only one guest found it to be comfortable.
ROUGE: 0.23999999507200007 BLEU: 0.029799893137327636

Processing dialogue 2
No response!
Processing dialogue 3


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
Yes, the Bridge Guest House is among the 5 options. It is known for its cozy beds and comfortable accommodations. If you're looking for a balance between price, comfort, and amenities, the Bridge Guest House is a good choice. Let me know if you need further details!

GROUND TRUTH: Some of the past guests say that the bed is really comfortable while others say it is very uncomfortable.
ROUGE: 0.11320754295478834 BLEU: 0.01034508119066299

Processing dialogue 4


...

MODEL: 
The Bridge Guest House does offer cozy beds. It is a 3-star hotel with free parking and free WiFi, making it a suitable option for your needs.

GROUND TRUTH: Some of the past guests say that the bed is really comfortable while others say it is very uncomfortable.
ROUGE: 0.0952380905215422 BLEU: 0.015266081935888859

Processing dialogue 4


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
The staff at Alexander Bed and Breakfast is friendly, polite, and responsive. They are always available to assist with any requests. If you have any more questions or need further assistance, feel free to ask!

GROUND TRUTH: The Alexanders gets some good reviews for the staff. Guests have said they're a joy to interact with, very friendly and helpful. Would you like to know more about them?
ROUGE: 0.2295081917334051 BLEU: 0.01700269763432147

Processing dialogue 5


It is recommended to enable `effective_order` for sentence-level BLEU.


MODEL: 
I cannot confirm if Kymmoy offers good views. You may want to check directly with the establishment for the most accurate information. Would you like me to book a table for you at Kymmoy? 

GROUND TRUTH: Yes they do have an incredible view which is amazing .The location is also nice and they do have amazing décor.
ROUGE: 0.0 BLEU: 0.01184949136625139

Mean BLEU 0.017612539512766725
Mean ROUGE 0.11677649731773007
BERTscore 0.8601757526397705
--------------------------------------------------------------------------------------------------------------------------------------------




## Temperature 0

### EX0-1.
    Mean BLEU 0.0318982427806526
    Mean ROUGE 0.19109908503699205
    BERTscore 0.8702526390552521
### EX0-2.
    Mean BLEU 0.03396629598866987
    Mean ROUGE 0.1610088238033507
    BERTscore 0.875411257147789
### EX0-3.
    Mean BLEU 0.032046169136594166
    Mean ROUGE 0.1653920041122659
    BERTscore 0.8756257772445679

## Temperature 1

### EX1-1.
    Mean BLEU 0.014016048906067786
    Mean ROUGE 0.11897074794437124
    BERTscore 0.861987034479777
### EX1-2.
    Mean BLEU 0.0217373160352085
    Mean ROUGE 0.1699494904727284
    BERTscore 0.8678601235151291
### EX1-3.
    Mean BLEU 0.017612539512766725
    Mean ROUGE 0.11677649731773007
    BERTscore 0.8601757526397705

## Analysis

### 
